## Packages

In [ ]:
# Basic packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Validation 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

## ANN - Multilayer Perceptron

### Pre-processing
Wir verwenden für diesen Teil wieder die Grundwasserqualitätsdaten, die wir bereits in Termin 4 für die Binär-Klassifikation verwendet haben. Ziel ist es, mit Hilfe der anderen Parameter jeweils die Landnutzung (d.h. landwirtschaftlich ja/nein) in Form einer Binär-Klassifikation sowie die Nitrat-Gehalte in Form einer Regression vorherzusagen.

Die Daten sind bereits teilweise pre-processed (kategorische Variablen sind encodiert, Werte unter der NWG wurden ersetzt). Wir prüfen nun noch, ob Nan-Werte vorhanden sind und löschen anschließend noch nicht benötigte Spalten.

In [ ]:
# Einladen der Daten
filename = 'gwdata_2005.csv'
dataset = pd.read_csv(filename, delimiter=';')

# Prüfen auf Nan-Werte
dataset.isnull().sum()


In [ ]:
# Pre-processing

dataset_cleaned = dataset.dropna()

mydata = dataset_cleaned.drop(['GWNum','Messstelle','Rechtswert', 'Hochwert', 'Aquifer','Aquifer2','landuse'], axis=1)
mydata.head()

## Klassifikation
### Splitten der Daten

Das Aufteilen des Datensatzes erfolgt mit der Funktion "train_test_split" des Packages sklearn.

Hier kann mit dem Parameter "test_size" das Verhältnis festgelegt werden und mit dem Parameter "random_state" die Reproduzierbarkeit garantiert werden. Der Testdatensatz dient später dazu, zu evaluieren, wie gut das trainierte neuronale Netz generalisieren kann, d.h. Daten vorhersagen, die es vorher "noch nie gesehen hat".

Um Overfitting zu vermeiden und/oder das ANN hinsichtlich sog. Hyperparameter zu optimieren müssen wir die vorhandenen Daten weiter in einzelne Datensätze aufteilen. Je nach Zusammenhang bzw. Verwendung werden diese als Stopp-Set, Optimierungs-Set oder als Validation-Set bezeichnet. Diese können ebenfalls mit der Funktion train_test_split erzeugt werden.

Für den Anfang wollen wir lediglich einen weiteren Datensatz, um Overfitting zu vermeiden. Dieser wird später bei jedem Trainingsschritt parallel beobachtet. Das ursprüngliche Training-Set (zunächst als temp bezeichnet) teilen wir dazu erneut auf:

In [ ]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)

from sklearn.model_selection import train_test_split

temp, test = train_test_split(mydata, test_size=0.2, random_state=27)

train, val = train_test_split(temp, test_size=0.1, random_state=27)

print('Größe Trainings-Set: ', train.shape)
print('Größe Validation-Set: ', val.shape)
print('Größe Test-Set: ', test.shape)

### Skalieren

Für neuronale Netze müssen die Eingangsdaten skaliert werden. Je nach Datensatz und Fragestellung kann man die Daten entweder Normieren oder Standardisieren (siehe Übung Termin 2). Wir wählen hier die Normierung, damit unsere landuse_num Kategorien erhalten bleiben.

Der Scaler wird zunächst an die Daten des Trainings-Set gefittet. Anschließend werden sowohl Trainings-Set,  Validation-Set und Test-Set transformiert.

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train)

train = scaler.transform(train)
val = scaler.transform(val)
test = scaler.transform(test)

### Aufteilen der Daten in Input- und Zieldaten

In [ ]:
X_train = train[:, :-1] # for all but last column --> Input variables
y_train = train[:, -1] # for last column --> Target variable

X_val = val[:, :-1] # for all but last column --> Input variables
y_val = val[:, -1] # for last column --> Target variable

X_test = test[:, :-1] # for all but last column --> Input variables
y_test = test[:, -1] # for last column --> Target variable

### Aufbau und Training des ANN

Wir importieren zunächst weitere nötige Pakete, und lesen dann die Anzahl der Input-features aus. Anschließend erstellen wir ein einfaches ANN mit einer Eingabeschicht (welche automatisch auf Basis der Anzahl der Input-Features erzeugt wird), einer verdeckten Schicht mit 5 Neuronen und einer Ausgabeschicht mit einem Neuron für unseren Zielwert (Nitrat). Vollständig miteinerander verbundene Schichten (fully connected layers) werden in Keras als Dense-Layer bezeichnet.

Weitere Hyperparameter des Modells (neben der Anzahl der Schichten und Neuronen) sind die Initialisierungsfunktion, die Aktivierungsfunktion, der Optimizer und die Batch-Size (Anzahl der Inputs, die dem Netz pro Epoche auf einmal präsentiert werden). Diese werden zunächst relativ wahllos gewählt. Als Aktivierungsfunktion verwenden wir "ReLu" für die verdeckte Schicht, was für die meisten Problem in der Regel gut funktioniert. Für die Ausgabeschicht verwenden wir eine sigmoidale Aktivierungsfunktion, um sicherzustellen, dass die Ausgabe des Netzwerks zwischen 0 und 1 liegt und sich leicht auf eine der beiden Klassen mit einem Standard-Schwellenwert von 0,5 abbilden lässt. Der Adam-Optimizer gilt ebenfalls für viele Probleme als Methode der Wahl. Bei der Batch-Size findet man häufig Anfangswerte von 16, 32 oder 64. Alle Hyperparameter können später "optimiert" werden. Als loss-Funktion kommt für die binäre Klassifikation "binary_crossentropy" zum Einsatz, als Metrik die "accuracy". Zunächst starten wir mit 50 Epochen.

Nach dem eigentlichen Training (model.fit) führen wir mit model.evaluate eine Vorhersage mit den validation-Inputs aus, um einen Hinweis dafür zu bekommen, wie gut das Modell ist.

In [ ]:
# import packages
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers
from matplotlib import pyplot

# determine the number of input features
n_features = X_train.shape[1]

# set initializer
kernel_initializer = initializers.glorot_uniform()

# define model
model = Sequential()
model.add(Dense(5, activation='relu', kernel_initializer=kernel_initializer, input_shape=(n_features,)))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1, validation_data=(X_val,y_val))

# make class predictions with the model
predictions = model.predict_classes(X_val)

# compute accuracy
_, accuracy = model.evaluate(X_val,y_val)
print('Accuracy: %.2f' % (accuracy*100))

# plot loss learning curves

pyplot.figure(figsize=(8,8)) 
pyplot.subplot(211)
pyplot.title('Cross-Entropy Loss', pad=-40)
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy learning curves
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

Falls wir sehen, dass ein Overfitting eintritt, können wir die Anzahl der Epochen von Hand anpassen und einen zweiten Durchlauf starten (später mehr zu Techniken, wie man das Training im Fall von Overfitting automatisch stoppen kann).

Die Accuracy beträgt bei diesem sehr einfachen und nicht optimierten Modell für die Validierungsdaten immerhin etwa 66% und ist damit in etwa vergleichbar mit der Performance des SVM/Scaled SVM aus Termin 4 (aber nicht so gut wie Random Forest). Da diese Konfiguration ganz gut zu funktionieren scheint, bauen wir das Modell nochmals neu auf, trainieren es mit dem kompletten Trainingsdatensatz (inklusive der Validierungs-Daten) und sagen damit die Landnutzung unseres Test-Datensatzes voraus.

In [ ]:
seed(1)
tf.random.set_seed(2)

# Split data
train, test = train_test_split(mydata, test_size=0.2, random_state=27)

# Scale data
scaler = MinMaxScaler()
scaler.fit(train)

train = scaler.transform(train)
test = scaler.transform(test)

# Input- and Target-Data
X_train = train[:, :-1] # for all but last column --> Input variables
y_train = train[:, -1] # for last column --> Target variable

X_test = test[:, :-1] # for all but last column --> Input variables
y_test = test[:, -1] # for last column --> Target variable


# determine the number of input features
n_features = X_train.shape[1]

# set initializer
kernel_initializer = initializers.glorot_uniform()


# define model
model = Sequential()
model.add(Dense(5, activation='relu', kernel_initializer=kernel_initializer, input_shape=(n_features,)))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

# make class predictions with the model
#predictions = model.predict_classes(X_val)

# compute accuracy
_, accuracy = model.evaluate(X_test,y_test)
print('Accuracy: %.2f' % (accuracy*100))


Auch dies scheint offensichtlich ähnlich gut zu funktionieren, die Vorhersage-Genauigkeit für den Test-Datensatz beträgt in diesem Durchlauf etwa 66 %. Die eigentlichen Daten können je nach Durchlauf aufgrund der unterschiedlichen Initialisierungen variieren. Wie man das Modell robuster macht, dazu bei der Regression mehr.

## Regression

### Splitten der Daten

Zunächst sortieren für die Regression nun die Spalten so um, dass die Spalte mit den Werten, die wir später vorhersagen wollen, in diesem Fall Nitrat, am Ende steht. Das macht uns  die Aufteilung in Input- und Zieldaten leichter. Anschließend wiederholen wir die  Skalierung und Aufteilung der Daten.

Das Aufteilen des Datensatzes erfolgt mit der Funktion "train_test_split" des Packages sklearn.

Hier kann mit dem Parameter "test_size" das Verhältnis festgelegt werden und mit dem Parameter "random_state" die Reproduzierbarkeit garantiert werden. Der Testdatensatz dient später dazu, zu evaluieren, wie gut das trainierte neuronale Netz generalisieren kann, d.h. Daten vorhersagen, die es vorher "noch nie gesehen hat".

Um Overfitting zu vermeiden und/oder das ANN hinsichtlich sog. Hyperparameter zu optimieren müssen wir die vorhandenen Daten weiter in einzelne Datensätze aufteilen. Je nach Zusammenhang bzw. Verwendung werden diese als Stopp-Set, Optimierungs-Set oder als Validation-Set bezeichnet. Diese können ebenfalls mit der Funktion train_test_split erzeugt werden.

Für den Anfang wollen wir lediglich einen weiteren Datensatz, um Overfitting zu vermeiden. Dieser wird später bei jedem Trainingsschritt parallel beobachtet. Das ursprüngliche Training-Set (zunächst als temp bezeichnet) teilen wir dazu erneut auf.



In [ ]:
mydata_reg = mydata[['Ca','Cl','DOC','LF_mS_m','K','Mg','Na','pH','SO4','SiO2','U','PSM','Aquifer_kz','landuse_num','NO3']]
mydata_reg.head()

In [ ]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)

from sklearn.model_selection import train_test_split

temp, test = train_test_split(mydata, test_size=0.2, random_state=27)

train, val = train_test_split(temp, test_size=0.1, random_state=27)

print('Größe Trainings-Set: ', train.shape)
print('Größe Validation-Set: ', val.shape)
print('Größe Test-Set: ', test.shape)

### Skalieren

Für neuronale Netze müssen die Eingangsdaten skaliert werden. Je nach Datensatz und Fragestellung kann man die Daten entweder Normieren oder Standardisieren (siehe Übung Termin 2). Wir wählen hier die Standardisierung, die für Regressionsproblem gut geeignet ist.

Der Scaler wird zunächst an die Daten des Trainings-Set gefittet. Anschließend werden sowohl Trainings-Set,  Validation-Set und Test-Set transformiert.

In [ ]:
scaler = StandardScaler()
scaler.fit(train)

train = scaler.transform(train)
val = scaler.transform(val)
test = scaler.transform(test)

### Aufteilen der Daten in Input- und Zieldaten

In [ ]:
X_train = train[:, :-1] # for all but last column --> Input variables
y_train = train[:, -1] # for last column --> Target variable

X_val = val[:, :-1] # for all but last column --> Input variables
y_val = val[:, -1] # for last column --> Target variable

X_test = test[:, :-1] # for all but last column --> Input variables
y_test = test[:, -1] # for last column --> Target variable

### Aufbau und Training des ANN

Wir importieren zunächst weitere nötige Pakete, und lesen dann die Anzahl der Input-features aus. Anschließend erstellen wir ein einfaches ANN mit einer Eingabeschicht (welche automatisch auf Basis der Anzahl der Input-Features erzeugt wird), einer verdeckten Schicht mit 5 Neuronen und einer Ausgabeschicht mit einem Neuron für unseren Zielwert (Nitrat).

Weitere Hyperparameter des Modells (neben der Anzahl der Schichten und Neuronen) sind die Initialisierungsfunktion, die Aktivierungsfunktion, der Optimizer und die Batch-Size (Anzahl der Inputs, die dem Netz pro Epoche auf einmal präsentiert werden). Diese werden zunächst relativ wahllos gewählt. Als Aktivierungsfunktion verwenden wir für die verdeckte Schicht "ReLu", was für Regressionsprobleme in der Regel gut funktioniert. Für die Ausgabeschicht wird keine Aktivierungsfunktion verwendet, da es sich um ein Regressionsproblem handelt und wir daran interessiert sind, numerische Werte direkt ohne Transformation vorherzusagen.

Der Adam-Optimizer gilt ebenfalls für viele Probleme als Methode der Wahl. Bei der Batch-Size findet man häufig Anfangswerte von 16, 32 oder 64. Alle Hyperparameter können später "optimiert" werden.

Als loss-Funktion kommt für die Regression der mean squared error zum Einsatz. Die Anzahl der Epochen erhöhen wir zunächst auf 100, da eine Regression ein komplexeres Problem darstellt, das evtl. mehr Training benötigt.

Nach dem eigentlichen Training (model.fit) führen wir eine Vorhersage mit den validation-Inputs aus, um einen Hinweis dafür zu bekommen, wie gut das Modell ist. Dazu vergleichen wir die Vorhersage mit den bekannten Zielwerten und berechnen den MSE.

In [ ]:
# import packages
from sklearn.metrics import mean_squared_error
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers
from matplotlib import pyplot

seed(1)
tf.random.set_seed(2)

# determine the number of input features
n_features = X_train.shape[1]

# set initializer
kernel_initializer = initializers.glorot_uniform()

# define model
model = Sequential()
model.add(Dense(5, activation='relu', kernel_initializer=kernel_initializer, input_shape=(n_features,)))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# fit the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=3, validation_data=(X_val,y_val))

# predict test set
yhat = model.predict(X_val)

# evaluate predictions
score = mean_squared_error(y_val, yhat)
print('MSE: %.3f' % score)

# plot learning curves
pyplot.title('Learning Curves')
pyplot.xlabel('Epoch')
pyplot.ylabel('MSE')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()

Wir sehen, dass ab etwa 60 Epochen ein Overfitting eintritt. Da dies aber sehr stark von den tatsächlichen Eingangsdaten sowie der Initialisierung abhängt, ist es besser, keine absolute Zahl festzulegen, sondern die Technik des "Early Stopping" anzuwenden. Dabei werden nach jeder Epoche die Gewichte abgespeichert und sobald sich der Fehler im Validierungsdatensatz für eine bestimmte Anzahl von Epochen nicht mehr verbessert, bricht das Training ab (Early Stopping) und springt zu den gespeicherten Gewichten des "besten" Durchlaufs zurück.

Im Folgenden der Code von oben mit Early Stopping. Dabei empfiehlt es sich, die Anzahl der Epochen etwas höher zu setzen, damit das Early Stopping auch greifen kann.

In [ ]:
# import packages
from sklearn.metrics import mean_squared_error
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot

seed(1)
tf.random.set_seed(2)

# determine the number of input features
n_features = X_train.shape[1]

# set initializer
kernel_initializer = initializers.glorot_uniform()

# patient early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)

# define model
model = Sequential()
model.add(Dense(5, activation='relu', kernel_initializer=kernel_initializer, input_shape=(n_features,)))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# fit the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1, validation_data=(X_val,y_val),
                    callbacks=[es])

# predict test set
yhat = model.predict(X_val)

# evaluate predictions
score = mean_squared_error(y_val, yhat)
print('MSE: %.3f' % score)

# plot learning curves
pyplot.title('Learning Curves')
pyplot.xlabel('Epoch')
pyplot.ylabel('MSE')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()

### kfold Crossvalidation
Um nun zu testen, wie robust die Vorhersage ist, wiederholen wir das Ganze mit einer kfold crossvalidation, d.h. wir splitten unsere Daten in jeweils 90% Trainings- und 10% Validierungsdaten. Dafür müssen wir die Reihenfolge des Datensplitting und der Skalierung ggü. oben etwas umstellen. Das Training und die Validation werden anschließend in einer Schleife ausgeführt.

In [ ]:
seed(1)
tf.random.set_seed(2)

temp, test = train_test_split(mydata_reg, test_size=0.2, random_state=27)

scaler = StandardScaler()
scaler.fit(temp)

temp = scaler.transform(temp)
test = scaler.transform(test)

X, y = temp[:, :-1], temp[:, -1]

X_test, y_test = test[:, :-1], test[:, -1]


# prepare cross validation
from numpy import mean
from numpy import std

kfold = KFold(10, shuffle=True, random_state=27)

# enumerate splits
scores = list()


for train_ix, val_ix in kfold.split(X, y):
    # split data
    X_train, X_val, y_train, y_val = X[train_ix], X[val_ix], y[train_ix], y[val_ix]
    # determine the number of input features
    n_features = X.shape[1]
    # set initializer
    kernel_initializer = initializers.glorot_uniform
    
    # patient early stopping
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)

    # define model
    model = Sequential()
    model.add(Dense(5, activation='relu', kernel_initializer=kernel_initializer, input_shape=(n_features,)))
    model.add(Dense(1))

    # compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # fit the model
    history = model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=3, validation_data=(X_val,y_val),
                       callbacks=[es])

    # predict test set
    yhat = model.predict(X_val)
    score = mean_squared_error(y_val, yhat)
    print('MSE: %.3f' % score)
    scores.append(score)
    
    # plot learning curves
    pyplot.title('Learning Curves')
    pyplot.xlabel('Epoch')
    pyplot.ylabel('MSE')
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='val')
    pyplot.legend()
    pyplot.show()


# summarize all scores
print('Mean MSE: %.3f (%.3f)' % (mean(scores), std(scores)))

Wir sehen, dass sich sowohl der Verlauf der Lernkurven als auch die Performance (also der MSE) der einzelnen Durchläufe zum Teil stark unterscheiden kann, was zum Einen von den unterschiedlichen Initialisierungen und zum Anderem von der Aufteilung des Datensatzes abhängt.

Auch bei einer zufälligen Aufteilung kann es sein, dass "zufällig" mehr schwierig vorherzusagende Werte im Validierungsatz landen. Dies sind vor allem Werte, die in der Verteilung der Trainingsdaten seltener vorkommen. Im Beispiel von Nitrat wären das hohe Messwerte. Daher ist es wichtig, mit einer kfold-Crossvalidation die mittlere Performance zu betrachten.

Dem Problem der Initialisierung begegnet man in der Regel damit, dass man viele Modelle mit unterschiedlichen Initialisierungen rechnet (also ein Ensemble bildet) und dann jeweils die mittlere Vorhersage aller Modelle betrachtet.

### Optimierung von Hyperparametern (der "einfache" Weg)

Ein einfacher Weg, Hyperparameter wie die Batchsize, Lernregel usw. zu optimieren, ist es, einfach alle möglichen Kombinationen durchzuprobieren. Dies wird als "grid search" bezeichnet. Der Vorteil ist, dass man auf jeden Fall die beste Variante finden wird, der Nachteil ist, dass bei einer größeren Anzahl von zu optimierenden Hyperparametern und deren jeweils mögliche Werte(-bereiche) die Anzahl der möglichen Kombinationen schnell so groß wird, dass eine grid search zeitlich gar nicht mehr möglich ist. Es gibt aber noch weitere, effizientere Methoden, dazu in einer späteren Vorlesung mehr.

Im Folgenden führen wir eine kleine grid search durch, wobei wir zwei weitere Optimizer, drei verschiedene Epochen-Anzahlen und drei verschiedene Batch-Sizes testen (also 27 mögliche Kombinationen). Schon das könnte je nach Rechner ganz schön lange dauern. Falls es zu lange dauert, können wir die Anzahl der zu optimierenden Parameter reduzieren.


In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

seed(1)
tf.random.set_seed(2)

def baseline_model(optimizer='adam'):
   
    # define model
    model = Sequential()
    model.add(Dense(5, activation='relu', kernel_initializer='glorot_uniform', input_shape=(n_features,)))
    model.add(Dense(1))

    # compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    return model

# import, split and scale data
temp, test = train_test_split(mydata_reg, test_size=0.2, random_state=27)

scaler = StandardScaler()
scaler.fit(temp)

temp = scaler.transform(temp)
test = scaler.transform(test)

X, y = temp[:, :-1], temp[:, -1]

    
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=16, verbose=1)

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam', 'SGD']
epochs = [30, 50, 70]
batches = [8, 16, 32]

param_grid = dict(optimizer=optimizers,
epochs=epochs,
batch_size=batches)

grid = GridSearchCV(estimator=estimator, param_grid=param_grid, scoring='neg_mean_squared_error')
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### Vorhersage der Test-Daten mit dem besten Modell

Jetzt wo wir wissen, was die beste Hyperparameter-Kombination ist, bauen wir das Modell nochmals neu auf, trainieren es mit dem kompletten Trainingsdatensatz (inklusive der Valididierungs-Daten) und sagen damit die Nitrat-Werte unseres Test-Datensatzes voraus. Ggf. weicht die "beste" Hyperparameter-Kombination von der hier gewählten ab.

Weiterhin nehmen wir die Skalierung getrennt nach Input- und Target-Daten vor, was später eine inverse Transformation erleichtert.

In [ ]:
seed(1)
tf.random.set_seed(2)

# Split data
train, test = train_test_split(mydata_reg, test_size=0.2, random_state=27)

# Input- and Target-Data
X_train = train.values[:, :-1] # for all but last column --> Input variables
y_train = train.values[:, -1] # for last column --> Target variable

X_test = test.values[:, :-1] # for all but last column --> Input variables
y_test = test.values[:, -1] # for last column --> Target variable

# reshape 1d arrays to 2d arrays

y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

# Scale data
scaler_input, scaler_target = StandardScaler(), StandardScaler()
scaler_input.fit(X_train)
scaler_target.fit(y_train)

X_train = scaler_input.transform(X_train)
y_train = scaler_target.transform(y_train)

X_test = scaler_input.transform(X_test)
y_test = scaler_target.transform(y_test)


# determine the number of input features
n_features = X_train.shape[1]


# define model
model = Sequential()
model.add(Dense(5, activation='relu', kernel_initializer='glorot_uniform', input_shape=(n_features,)))
model.add(Dense(1))

# compile the model
model.compile(optimizer='SGD', loss='mean_squared_error')

# fit/train the model
history = model.fit(X_train, y_train, epochs=70, batch_size=32, verbose=1)

# predict Test data
yhat = model.predict(X_test)

# Scores
score = mean_squared_error(y_test, yhat)
print('MSE: %.3f' % score)

### Vergleich vorhergesagte und tatsächliche Daten

Um die vorhergesagten und tatsächlichen Daten zu vergleichen, können wir nun weitere Fehlermaße berechnen (z.B. MSE, RMSE, R²) sowie die Daten in einem Scatterplot darstellen. Dazu re-transformieren wir die Zieldaten zunächst wieder auf die ursprünglichen Werte.


In [ ]:
import matplotlib.pyplot as plt

# inverse transform
y_test = scaler_target.inverse_transform(y_test)
yhat = scaler_target.inverse_transform(yhat)

# Scores
mse = mean_squared_error(y_test, yhat)
r2 = r2_score(y_test, yhat)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % np.sqrt(mse))
print('R_squared: %.3f' % r2)

plt.figure()
plt.scatter(y_test, yhat)
plt.plot(y_test, y_test, color = 'red', label = 'x=y')
plt.xlabel('actual')
plt.ylabel('predicted')
plt.legend()
plt.show()

Naja, nicht ganz perfekt :-) Vor allem die höheren Werte scheinen schwieriger vorherzusagen zu sein. Das liegt daran, dass diese auch im Trainingsdatensatz seltener vorkommen (ein bekanntes Problem, was als "imbalanced regression/classification" bezeichnet wird).

Eine weitere Optimierung könnte man durch folgende Punkte erreichen:

* Die Aquiferkennzahl ist eigentlich eine kategorische Variable, wurde bisher aber ordinal encodiert. Hier könnte man ein One-Hot- oder Target-Encoding testen.
* Vermutlich korrelieren einige der Input-Variablen recht stark miteinander. Durch eine vorgeschaltete PCA könnten sich die Vorhersagen ebenfalls verbessern.
* Die Hyperparameter könnten noch besser optimiert werden, d.h. mehr Optionen für die bereits getesteten Hyperparameter oder weitere wie die Aktivierungsfunktion, Initialisierung und Lernrate.
* Man könnte die Anzahl der Neuronen und die Anzahl der Schichten verändern.

## Übungsaufgabe 1

Verändern Sie die Anzahl der Neuronen in der verdeckten Schicht! Sie können dies ebenfalls mit einer GridSearchCV machen, in dem Sie einen Parameter "neurons" im dictionary param_grid (s.o.) definieren und für diesen verschiedene Anzahlen als Optionen vorgeben. Damit die GridSearch nicht zu lange dauert, sollten sie einen oder zwei der anderen Optimierungsparameter dafür rausnehmen. Experimentieren Sie außerdem noch mit einer zweiten verdeckten Schicht und optimieren Sie in dieser ebenfalls die Anzahl der Neuronen.
Achtung: Bei der Definition des baseline_model müssen sie für Parameter, die später über das param_dict definiert werden, einen Anfangswert eingeben!

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

def baseline_model(optimizer='adam', neurons=5):
   
    # define model
    model = Sequential()
    model.add(Dense(neurons, activation='relu', kernel_initializer='glorot_uniform', input_shape=(n_features,)))
    model.add(Dense(1))

    # compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    return model

# import, split and scale data

# ...

## Übungsaufgabe 2

Da sich mit einer veränderten Zahl der Neuronen und/oder Schichten wiederum andere Hyperparameter, vor allem die Anzahl der Epochen und der Batch-Size als günstiger erweisen könnten, ist es am besten, möglichst viele Hyperparameter gleichzeitig zu optimieren. Hier stößt die GridSearchCV aber schnell an ihre Grenzen. Eine weitere Alternative ist die RandomizedSearchCV, bei der eine definierte Anzahl an zufällig gewählten Kombination aus dem vorgegebenen Parameter-Space getestet werden. Hierbei ist zwar nicht garantiert, dass die absolut beste Kombination gefunden wird, jedoch liefert es meistens bei einem geringeren Rechenaufwand schon ganz brauchbare Hinweise, welche Kombinationen gut funktionieren und welche nicht.

Achtung: beim Befehl RandomizedSearchCV wird der Parameter "param_grid" durch "param_distributions" ersetzt. Ansonsten ist die Verwendung exakt gleich wie bei GridSearchCV. Als weitere Option kann man die Anzahl der Iterationen über "n_iter" festelegen. Default-Wert ist 10.

Optimieren Sie nun mehrere Hyperparameter gleichzeitig! Für den optimizer können Sie z.B. zusätzlich die Optionen 'Adagrad', 'Adadelta', 'Adamax' und 'Nadam' testen. Für die Aktivierungsfunktion kommen z.B. 'softmax', 'softplus', 'softsign', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear' in Frage und für die Initialisierung 'uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'. Je mehr Parameter und Optionen Sie gleichzeitig verwenden, desto höher sollten Sie den Wert für n_iter setzen, damit eine vernünftige Anzahl der möglichen Kombinationen ausprobiert wird. Als Daumenwert sollte dies mindestens etwa 0,1 * die Anzahl aller möglichen Kombinationen sein!

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

def baseline_model(optimizer='adam', neurons=5, activation='relu'):
   
    # define model
    model = Sequential()
    model.add(Dense(neurons, activation=activation, kernel_initializer='glorot_uniform', input_shape=(n_features,)))
    model.add(Dense(1))

    # compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    return model

# import, split and scale data
# ...


grid = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid, scoring='neg_mean_squared_error', n_iter=50)

# ...

Wenn Sie eine bessere Parameterkombination als in der Vorlesung verwendet finden, so schreiben Sie den Algorithmus zur Vorhersage der Test-Daten mit dem besten Modell entsprechend um und sagen Sie dann mit diesem wiederum die Test-Daten voraus. Wie haben sich hier die Ergebnisse verändert?

In [ ]:
# Use optimized model for prediction of test data

# ...

## Übungsaufgabe 3

Wenden Sie die gelernten Methoden des Kfold Crossvalidation und der Hyperparameter-Optimierung auch auf die Binär-Klassifikation an und schauen Sie, ob Sie die Performance des Random Forest aus Termin 4 übertreffen können.

In [ ]:
# ...

In [ ]:
# Use optimized model for prediction of test data

# ...

## Übungsaufgabe 4

Wenden Sie nun die hier gelernten Methoden zur Regression mit einem MLP auf den Beton-Datensatz aus Termin 4 (Concrete_Data.csv) an! Wie sind die Ergebnisse, die Sie erzielen können im Vergleich zum in Termin 4 angewandten Random-Forest Methode?

In [ ]:
# ...

In [ ]:
# Use optimized model for prediction of test data

# ...

## Übungsaufgabe 5

Testen Sie, ob Sie für den Grundwasserqualitätsdatensatz die Vorhersage von Nitrat verbessern können, wenn Sie eine PCA vorschalten und/oder den Aquifer anders codieren!